In [2]:
# Import my modules.
import sys, os
from pathlib import Path
current_dir = os.path.join(Path().resolve())
sys.path.append(str(current_dir) + '/../')

from modules import utils
from modules import models
from modules import preprocess

import importlib
for m in [utils, models, preprocess]:
    importlib.reload(m)

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

train = pd.read_csv("../input/train.csv")
tr_unq_cfips = list(train.cfips.unique())
tr_unq_count = len(tr_unq_cfips)

#change this to your submission file
fname = "../submission/submission_2023-03-12_15_17_53.csv" 

df = pd.read_csv(fname)
print(len(df))

#err case testing
#df.loc[12540, 'microbusiness_density']=np.Inf
#df.loc[12540, 'microbusiness_density']=np.NaN
#df = df.append({'row_id':"1001_2023-03-01", 'microbusiness_density':np.NaN}, ignore_index = True)
#df = df.append({'row_id':"999444_2023-03-01", 'microbusiness_density':43}, ignore_index = True)
#df = df.drop(12540)

df = df.replace([np.inf, -np.inf], np.nan)
if len(df.dropna()) != len(df):
    print("Warning, inf or nan values in submission, count is", len(df) - len(df.dropna()))

if len(df.columns) != 2:
    print("Warning, should ideally be two columns is ", len(df.columns))

if df.columns[0] != 'row_id':
    print("Warning, first columns should ideally be 'row_id' is", df.columns[0])

if df.columns[1] != 'microbusiness_density':
    print("Warning, second column should ideally be 'microbusiness_density' is", df.columns[1])

df['first_day_of_month'] = df['row_id'].apply(lambda x:x[-10:])
df['cfips'] = df['row_id'].apply(lambda x:int(x[:x.index("_")]))

for d in ["2023-01-01", "2023-02-01", "2023-03-01","2023-04-01","2023-05-01", '2023-06-01']:
    print("Checking",d)
    dfd = df.query(f"first_day_of_month == '{d}'")

    sub_unq_cfips = list(dfd['cfips'].unique())
    sub_unq_count = len(sub_unq_cfips)
    if sub_unq_count != tr_unq_count:
        print(f"Invalid unique number of cfips for {d}, should be {tr_unq_count}", sub_unq_count)

    tset = set(sub_unq_cfips+tr_unq_cfips)
    tset_count = len(tset)
    if  tset_count != tr_unq_count:
        print(f"Invalid cfips are included for {d}, set size mismatch should be {tr_unq_count} is ", tset_count)

    l = len(dfd)
    if l != tr_unq_count:
        print(f"Invalid # of predictions for date {d}, should be {tr_unq_count} is ", l)

print("If no warnings were stated, all tests above passed.")

25080
Checking 2023-01-01
Checking 2023-02-01
Checking 2023-03-01
Checking 2023-04-01
Checking 2023-05-01
Checking 2023-06-01
If no warnings were stated, all tests above passed.
